In [1]:
import hopsworks
import pandas as pd

# Step 1: Login to Hopsworks and connect to the project
project = hopsworks.login()
fs = project.get_feature_store()

print("✅ Connected to Hopsworks project and feature store")


/Users/snehitha/citi_bike_project/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-05-08 16:33:49,569 INFO: Initializing external client
2025-05-08 16:33:49,570 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 16:33:51,079 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215691
✅ Connected to Hopsworks project and feature store


In [3]:
# Read from materialized Hopsworks table using Hive
fg = fs.get_feature_group(name="citi_bike_features", version=2)
df = fg.read(read_options={"use_hive": True})  # ✅ fixes FlyingDuckException
print("✅ Loaded feature group with shape:", df.shape)
df.head()


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.63s) 
✅ Loaded feature group with shape: (26100, 54)


,target,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,...,lag_44,lag_45,lag_46,lag_47,lag_48,hour,dayofweek,is_weekend,location_id,pickup_hour
0,0,0.0,0.0,0.0,0.0,0.0,9.0,5.0,27.0,30.0,...,0.0,0.0,1.0,0.0,0.0,3,1,0,JC115,2024-10-01 03:00:00+00:00
1,7,8.0,4.0,6.0,6.0,5.0,4.0,5.0,1.0,0.0,...,1.0,1.0,4.0,1.0,3.0,20,0,0,JC115,2024-02-19 20:00:00+00:00
2,6,3.0,4.0,4.0,4.0,2.0,6.0,4.0,3.0,0.0,...,23.0,36.0,12.0,7.0,1.0,14,3,0,JC115,2024-09-05 14:00:00+00:00
3,1,3.0,3.0,3.0,6.0,14.0,19.0,29.0,45.0,17.0,...,1.0,1.0,3.0,2.0,3.0,1,1,0,JC115,2024-10-08 01:00:00+00:00
4,6,5.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,3.0,0.0,2.0,0.0,8,0,0,HB102,2024-12-02 08:00:00+00:00


In [15]:
# Example: get last timestamp per location (or one location for now)
target_location = "JC115"  # change to your actual string IDs like HB102, HB105
df_latest = df[df["location_id"] == target_location].sort_values("pickup_hour").tail(1)

# Keep lag features + time-based features
X_latest = df_latest[[col for col in df.columns if "lag_" in col or col in ["hour", "dayofweek", "is_weekend"]]]

print("✅ Prepared latest input for prediction:", X_latest.shape)
X_latest.head()


✅ Prepared latest input for prediction: (1, 51)


,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,...,lag_42,lag_43,lag_44,lag_45,lag_46,lag_47,lag_48,hour,dayofweek,is_weekend
21914,1.0,2.0,0.0,0.0,5.0,9.0,8.0,9.0,6.0,7.0,...,0.0,0.0,0.0,0.0,3.0,5.0,1.0,23,1,0


In [16]:
import mlflow
import mlflow.pyfunc

mlflow.set_tracking_uri("https://dagshub.com/sai-snehitha/citi-bike-prediction-system.mlflow")
mlflow.set_registry_uri("https://dagshub.com/sai-snehitha/citi-bike-prediction-system.mlflow")

model_name = "citi_bike_best_model"
model_uri = f"models:/{model_name}/latest"  # You can also use "/3" if latest doesn't work

# Load model
loaded_model = mlflow.pyfunc.load_model(model_uri=model_uri)

# Predict
prediction = loaded_model.predict(X_latest)
print("✅ Prediction:", prediction[0])


✅ Prediction: 1.423989151887564
